# Wheat Disease Classifier- PytorchAnnual Hackathon

# Wheat Disease Classification

![](https://cropscience.bayer.co.uk/media/86153/brown-rust_wheat_375x225_halfwidth.jpg)

## Background

Prominent diseases of wheat that currently contribute to yield losses include the rusts, blotches and head blight/scab. Other recently emerged or relatively unnoticed diseases, such as wheat blast and spot blotch, respectively, also threaten grain production. 

In [2]:
# importing core packages
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image

# importing torch and visual libraries
import seaborn as sns
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, utils, datasets
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from sklearn.metrics import classification_report, confusion_matrix
 
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import random, os
# Any results you write to the current directory are saved as output.

# Reading Dataset

In [3]:
sample_submission_df = pd.read_csv("/kaggle/input/cgiar-computer-vision-for-crop-disease/sample_submission.csv")
sample_submission_df.shape

In [4]:
# declaring the train classes and test directorires
train_dir = '/kaggle/input/cgiar-computer-vision-for-crop-disease/train/train'
test_dir = '/kaggle/input/cgiar-computer-vision-for-crop-disease/test/test'
train_leaf_rust_dir = '/kaggle/input/cgiar-computer-vision-for-crop-disease/train/train/leaf_rust'
train_stem_rust_dir = '/kaggle/input/cgiar-computer-vision-for-crop-disease/train/train/stem_rust'
train_healthy_wheat_dir = '/kaggle/input/cgiar-computer-vision-for-crop-disease/train/train/healthy_wheat'

In [5]:
# directory to store processed images
train_new_dir = '/kaggle/working/train'
test_new_dir = '/kaggle/working/test'
train_new_leaf_rust_dir = '/kaggle/working/train/leaf_rust'
train_new_stem_rust_dir = '/kaggle/working/train/stem_rust'
train_new_healthy_wheat_dir = '/kaggle/working/train/healthy_wheat'
os.mkdir(train_new_dir)
os.mkdir(test_new_dir)
os.mkdir(train_new_leaf_rust_dir)
os.mkdir(train_new_stem_rust_dir)
os.mkdir(train_new_healthy_wheat_dir)

# Preprocessing and Transformation

In [6]:
# function to process the raw images and stor in a new directory
def process_img(old_dir, new_dir):
    for dirname, _, filenames in os.walk(old_dir):
        dir_p = new_dir + '/'
        for filename in filenames:
            filename_name = filename.split('.')[0]
            filename_ext = filename.split('.')[-1].lower()
            if filename_ext != "gif":
                img = Image.open(dirname + '/' + filename)
            else:
                img = Image.open(dirname + '/' + filename).convert('RGB')
            path = dir_p + filename_name + '.jpeg'
            img.save(path)

In [7]:
process_img(train_leaf_rust_dir, train_new_leaf_rust_dir)
process_img(train_stem_rust_dir, train_new_stem_rust_dir)
process_img(train_healthy_wheat_dir, train_new_healthy_wheat_dir)

In [8]:
# defining image transformations for train and test data input
image_transforms = {
    "train": transforms.Compose([
        transforms.Resize((300, 300)),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5],
                             [0.5, 0.5, 0.5])
    ]),
    "test": transforms.Compose([
        transforms.Resize((300, 300)),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5],
                             [0.5, 0.5, 0.5])
    ])
}

In [9]:
wht_dataset = datasets.ImageFolder(root = train_new_dir ,
                                   transform = image_transforms["train"]
                                  )
wht_dataset

In [10]:
wht_dataset.class_to_idx


In [11]:
idx2class = {v: k for k, v in wht_dataset.class_to_idx.items()}
idx2class

In [12]:
def get_class_distribution(dataset_obj):
    count_dict = {k:0 for k,v in dataset_obj.class_to_idx.items()}
    for _, label_id in dataset_obj:
        label = idx2class[label_id]
        count_dict[label] += 1
    return count_dict
def plot_from_dict(dict_obj, plot_title, **kwargs):
    return sns.barplot(data = pd.DataFrame.from_dict([dict_obj]).melt(), x = "variable", y="value", hue="variable", **kwargs).set_title(plot_title)
plt.figure(figsize=(15,8))
plot_from_dict(get_class_distribution(wht_dataset), plot_title="Entire Dataset (before train/val/test split)")

In [13]:
wht_dataset_size = len(wht_dataset)
wht_dataset_indices = list(range(wht_dataset_size))

In [14]:
np.random.shuffle(wht_dataset_indices)
val_split_index = int(np.floor(0.2 * wht_dataset_size))


In [15]:
train_idx, val_idx = wht_dataset_indices[val_split_index:], wht_dataset_indices[:val_split_index]

In [16]:
# Sampling the train and Test data
train_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(val_idx)

In [17]:
test_new_dir='/kaggle/working/test/test'
os.mkdir(test_new_dir)

process_img(test_dir, test_new_dir)


In [18]:
wht_dataset_test = datasets.ImageFolder(root = '/kaggle/working/test/' ,
                                        transform = image_transforms["test"])
wht_dataset_test

In [19]:
# passing the test, train and sample data to the DataLoader

train_loader = DataLoader(dataset=wht_dataset, shuffle=False, batch_size=8, sampler=train_sampler)
val_loader = DataLoader(dataset=wht_dataset, shuffle=False, batch_size=1, sampler=val_sampler)
test_loader = DataLoader(dataset=wht_dataset_test, shuffle=False, batch_size=1)

In [20]:
def get_class_distribution_loaders(dataloader_obj, dataset_obj):
    count_dict = {k:0 for k,v in dataset_obj.class_to_idx.items()}
    if dataloader_obj.batch_size == 1:    
        for _,label_id in dataloader_obj:
            y_idx = label_id.item()
            y_lbl = idx2class[y_idx]
            count_dict[str(y_lbl)] += 1
    else: 
        for _,label_id in dataloader_obj:
            for idx in label_id:
                y_idx = idx.item()
                y_lbl = idx2class[y_idx]
                count_dict[str(y_lbl)] += 1
    return count_dict

In [21]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(18,7))
plot_from_dict(get_class_distribution_loaders(train_loader, wht_dataset), plot_title="Train Set", ax=axes[0])
plot_from_dict(get_class_distribution_loaders(val_loader, wht_dataset), plot_title="Val Set", ax=axes[1])

In [22]:
#declaring a single batch from Train sample data loader
single_batch = next(iter(train_loader))


In [23]:
single_batch[0].shape


In [24]:
single_image = single_batch[0][0]
single_image.shape

In [25]:
print("Output label tensors: ", single_batch[1])
print("\nOutput label tensor shape: ", single_batch[1].shape)


In [26]:
# Display an simage from a single batch
plt.imshow(single_image.permute(1, 2, 0))


In [27]:
single_batch_grid = utils.make_grid(single_batch[0], nrow=4)
plt.figure(figsize = (10,10))
plt.imshow(single_batch_grid.permute(1, 2, 0))

# Training and Testing the Image Classifier

In [28]:
# Defining classifier torch class
class WheatClassifier(nn.Module):
    def __init__(self):
        super(WheatClassifier, self).__init__()
        self.block1 = self.conv_block(c_in=3, c_out=256, dropout=0.1, kernel_size=5, stride=1, padding=2)
        self.block2 = self.conv_block(c_in=256, c_out=128, dropout=0.1, kernel_size=3, stride=1, padding=1)
        self.block3 = self.conv_block(c_in=128, c_out=64, dropout=0.1, kernel_size=3, stride=1, padding=1)
        self.lastcnn = nn.Conv2d(in_channels=64, out_channels=3, kernel_size=75, stride=1, padding=0)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
    def forward(self, x):
        x = self.block1(x)
        x = self.maxpool(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.maxpool(x)
        x = self.lastcnn(x)
        return x
    def conv_block(self, c_in, c_out, dropout, **kwargs):
        seq_block = nn.Sequential(
            nn.Conv2d(in_channels=c_in, out_channels=c_out, **kwargs),
            nn.BatchNorm2d(num_features=c_out),
            nn.ReLU(),
            nn.Dropout2d(p=dropout)
        )
        return seq_block

In [30]:
# creating an instance of the model class
model = WheatClassifier()

# declaring a device for model training
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model.to(device)
print(model)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.005)

In [31]:
# defining the model accuracy function during train
def multi_acc(y_pred, y_test):
    y_pred_softmax = torch.log_softmax(y_pred, dim = 1)
    _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)    
    correct_pred = (y_pred_tags == y_test).float()
    acc = correct_pred.sum() / len(correct_pred)
    acc = torch.round(acc * 100)
    return acc

In [32]:
accuracy_stats = {
    'train': [],
    "val": []
}
loss_stats = {
    'train': [],
    "val": []
}

In [33]:
print("Begin training.")
for e in tqdm(range(1, 11)):
    # TRAINING
    train_epoch_loss = 0
    train_epoch_acc = 0
    model.train()
    for X_train_batch, y_train_batch in train_loader:
        X_train_batch, y_train_batch = X_train_batch.to(device), y_train_batch.to(device)
        optimizer.zero_grad()
        y_train_pred = model(X_train_batch).squeeze()
        train_loss = criterion(y_train_pred, y_train_batch)
        train_acc = multi_acc(y_train_pred, y_train_batch)
        train_loss.backward()
        optimizer.step()
        train_epoch_loss += train_loss.item()
        train_epoch_acc += train_acc.item()
    # VALIDATION
    with torch.no_grad():
        model.eval()
        val_epoch_loss = 0
        val_epoch_acc = 0
        for X_val_batch, y_val_batch in val_loader:
            X_val_batch, y_val_batch = X_val_batch.to(device), y_val_batch.to(device)
            y_val_pred = model(X_val_batch).squeeze()
            y_val_pred = torch.unsqueeze(y_val_pred, 0)
            val_loss = criterion(y_val_pred, y_val_batch)
            val_acc = multi_acc(y_val_pred, y_val_batch)
            val_epoch_loss += train_loss.item()
            val_epoch_acc += train_acc.item()
    loss_stats['train'].append(train_epoch_loss/len(train_loader))
    loss_stats['val'].append(val_epoch_loss/len(val_loader))
    accuracy_stats['train'].append(train_epoch_acc/len(train_loader))
    accuracy_stats['val'].append(val_epoch_acc/len(val_loader))
    print(f'Epoch {e+0:02}: | Train Loss: {train_epoch_loss/len(train_loader):.5f} | Val Loss: {val_epoch_loss/len(val_loader):.5f} | Train Acc: {train_epoch_acc/len(train_loader):.3f}| Val Acc: {val_epoch_acc/len(val_loader):.3f}')

In [34]:
train_val_acc_df = pd.DataFrame.from_dict(accuracy_stats).reset_index().melt(id_vars=['index']).rename(columns={"index":"epochs"})
train_val_loss_df = pd.DataFrame.from_dict(loss_stats).reset_index().melt(id_vars=['index']).rename(columns={"index":"epochs"})

# Plot line curve charts
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(30,10))
sns.lineplot(data=train_val_acc_df, x = "epochs", y="value", hue="variable",  ax=axes[0]).set_title('Train-Val Accuracy/Epoch')
sns.lineplot(data=train_val_loss_df, x = "epochs", y="value", hue="variable", ax=axes[1]).set_title('Train-Val Loss/Epoch')

In [35]:
y_pred_list = []
y_true_list = []
with torch.no_grad():
    for x_batch, y_batch in tqdm(test_loader):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        y_test_pred = model(x_batch)
        _, y_pred_tag = torch.max(y_test_pred, dim = 1)
        y_pred_list.append(y_pred_tag.cpu().numpy())
        y_true_list.append(y_batch.cpu().numpy())

In [36]:
y_pred_list = [i[0][0][0] for i in y_pred_list]
y_true_list = [i[0] for i in y_true_list]

In [37]:
print(classification_report(y_true_list, y_pred_list))


# Saving and loading the torch model**

In [38]:
PATH_OP = 'wheatDisease.pth'


torch.save(model.state_dict(), PATH_OP)

# Model class must be defined somewhere
model2 = WheatClassifier()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model2.to(device)
model2.load_state_dict(torch.load(PATH_OP))

model2.eval()

In [39]:
y_pred_list = []
y_true_list = []
with torch.no_grad():
    for x_batch, y_batch in tqdm(test_loader):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        y_test_pred = model2(x_batch)
        _, y_pred_tag = torch.max(y_test_pred, dim = 1)
        y_pred_list.append(y_pred_tag.cpu().numpy())
        y_true_list.append(y_batch.cpu().numpy())

In [40]:
y_pred_list = [i[0][0][0] for i in y_pred_list]
y_true_list = [i[0] for i in y_true_list]

In [41]:
print(classification_report(y_true_list, y_pred_list))


# Thank You